#### Import block

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

%matplotlib inline

In [2]:
# Check for available GPU.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Function definitions

In [3]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_input(path):
    # Load array.
    t_input = np.load(path)
    # Pad to even number of pixels
    t_input = np.pad(t_input, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    t_input = tf.expand_dims(t_input, axis = -1)
    return t_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    # Pad to even number of pixels
    #t_output = np.pad(t_output, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    #t_output = tf.expand_dims(t_output, axis = -1)
    t_output = t_output.sum()
    return t_output

def data_generator(samples, batch_size = 64):
    
    while True:
        # Select files (paths/indices) for the batch
        batch_samples  = np.random.choice(a = samples.index, 
                                      size = batch_size)
        batch_input  = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for sample in batch_samples:
          input = get_input(samples.loc[sample].features)
          output = get_output(samples.loc[sample].labels)

          batch_input += [input]
          batch_output += [output]
        # Return a tuple of (input, output) to feed the network
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        
        yield(batch_x, batch_y)


In [4]:
m = pd.read_csv('Sample_Dataset/train/meta.csv')
m

,Unnamed: 0,Lon,Lat,Date,features,labels
0,356962,-53.345535,-6.535028,2017-07-30,Sample_Dataset/train/features/356962.npy,Sample_Dataset/train/labels/356962.npy
1,546517,-47.329685,-8.260676,2017-08-20,Sample_Dataset/train/features/546517.npy,Sample_Dataset/train/labels/546517.npy
2,799359,-50.967861,-8.255809,2017-09-04,Sample_Dataset/train/features/799359.npy,Sample_Dataset/train/labels/799359.npy
3,714590,-66.144379,-12.624272,2017-08-31,Sample_Dataset/train/features/714590.npy,Sample_Dataset/train/labels/714590.npy
4,240012,-48.350338,-11.838207,2017-07-14,Sample_Dataset/train/features/240012.npy,Sample_Dataset/train/labels/240012.npy
...,...,...,...,...,...,...
6491,173827,-54.326607,-11.983384,2017-06-20,Sample_Dataset/train/features/173827.npy,Sample_Dataset/train/labels/173827.npy
6492,1377764,-47.425915,-7.486426,2017-09-26,Sample_Dataset/train/features/1377764.npy,Sample_Dataset/train/labels/1377764.npy
6493,1444041,-65.172768,-10.714162,2017-10-02,Sample_Dataset/train/features/1444041.npy,Sample_Dataset/train/labels/1444041.npy
6494,644793,-53.701656,-15.977278,2017-08-27,Sample_Dataset/train/features/644793.npy,Sample_Dataset/train/labels/644793.npy


#### Load data

In [7]:
# Get data from one chip
sample_input = np.load('Sample_Dataset/train/features/3243.npy')

# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
a = tf.expand_dims(a, axis = -1)
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)
# Display shape for verification.
a.shape
sample_input.shape

(4, 31, 31)

#### Experiment with basic model structure

In [4]:
b = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           input_shape=(a))(a)
b.shape

TensorShape([1, 4, 32, 32, 64])

In [5]:
c = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
c.shape

TensorShape([1, 4, 32, 32, 64])

In [6]:
d = layers.MaxPooling3D((2,2,2))(c)
d.shape

TensorShape([1, 2, 16, 16, 64])

In [9]:
e = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='relu')(d)
e.shape

TensorShape([1, 2, 16, 16, 128])

In [10]:
f = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='relu')(e)
f.shape

TensorShape([1, 2, 16, 16, 128])

In [11]:
g = layers.MaxPooling3D((1,2,2))(f)
g.shape

TensorShape([1, 2, 8, 8, 128])

In [12]:
h = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='relu')(g)
h.shape

TensorShape([1, 2, 8, 8, 256])

In [13]:
i = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='relu')(h)
i.shape

TensorShape([1, 2, 8, 8, 256])

In [14]:
j = layers.MaxPooling3D((1,2,2))(i)
j.shape

TensorShape([1, 2, 4, 4, 256])

In [15]:
k = tf.keras.layers.Conv3D(512, (3,3,3), padding = 'same', activation='relu')(j)
k.shape

TensorShape([1, 2, 4, 4, 512])

In [16]:
l = tf.keras.layers.Conv3D(512, (3,3,3), padding = 'same', activation='relu')(k)
l.shape

TensorShape([1, 2, 4, 4, 512])

In [17]:
m = layers.MaxPooling3D((2,4,4))(l)
m.shape

TensorShape([1, 1, 1, 1, 512])

In [18]:
n = layers.Flatten()(m)
n.shape

TensorShape([1, 512])

In [19]:
o = layers.Dense(64, activation='relu')(n)
o.shape

TensorShape([1, 64])

In [20]:
p = layers.Dense(32, activation='relu')(o)
p.shape

TensorShape([1, 32])

In [21]:
q = layers.Dense(16, activation='relu')(p)
q.shape

TensorShape([1, 16])

In [22]:
r = layers.Dense(1, activation='linear')(q)
r.shape

TensorShape([1, 1])

#### Model assembly

In [23]:
inputs = layers.Input(shape=((4,32,32,1)))
b = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='gelu', bias_initializer=Constant(0.01))(inputs)
c = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='gelu', bias_initializer=Constant(0.01))(b)
d = layers.MaxPooling3D((2,2,2))(c)
d = layers.BatchNormalization()(d)
d = layers.Dropout(0.3)(d)
e = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='gelu')(d)
f = tf.keras.layers.Conv3D(64, (3,3,3), padding = 'same', activation='gelu')(e)
g = layers.MaxPooling3D((1,2,2))(f)
g = layers.BatchNormalization()(g)
g = layers.Dropout(0.3)(g)
h = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='gelu')(g)
i = tf.keras.layers.Conv3D(128, (3,3,3), padding = 'same', activation='gelu')(h)
j = layers.MaxPooling3D((1,2,2))(i)
j = layers.BatchNormalization()(j)
j = layers.Dropout(0.3)(j)
k = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='gelu')(j)
l = tf.keras.layers.Conv3D(256, (3,3,3), padding = 'same', activation='gelu')(k)
m = layers.MaxPooling3D((2,4,4))(l)
m = layers.BatchNormalization()(m)
m = layers.Dropout(0.3)(m)
n = layers.Flatten()(m)
o = layers.Dense(1024, activation='gelu')(n)
o = layers.BatchNormalization()(o)
o = layers.Dropout(0.3)(o)
p = layers.Dense(1024, activation='gelu')(o)
p = layers.BatchNormalization()(p)
p = layers.Dropout(0.3)(p)
q = layers.Dense(1024, activation='gelu')(p)
q = layers.BatchNormalization()(q)
q = layers.Dropout(0.3)(q)
outputs = layers.Dense(1, activation='linear')(q)

In [24]:
forecast_model = tf.keras.Model(inputs, outputs, name="3D_CNN")

In [25]:
# Verify output shape.
forecast_model.predict(a).shape

1/1 [==============================] - 0s 203ms/step


(1, 1)

In [26]:
# Display model details.
forecast_model.summary()

Model: "3D_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 32, 32, 1)]    0         
                                                                 
 conv3d_8 (Conv3D)           (None, 4, 32, 32, 32)     896       
                                                                 
 conv3d_9 (Conv3D)           (None, 4, 32, 32, 32)     27680     
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 2, 16, 16, 32)    0         
 3D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 2, 16, 16, 32)    128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2, 16, 16, 32)     0    

In [27]:
# Compile model.
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
forecast_model.compile(loss = tf.keras.losses.MeanSquaredError(), 
                       optimizer=opt, 
                       metrics = [tf.keras.metrics.MeanSquaredError()])

In [28]:
# Data loaders.
batch_size = 64
meta_t = pd.read_csv('Sample_Dataset/train/meta.csv')
meta_v = pd.read_csv('Sample_Dataset/val/meta.csv')
t_gen = data_generator(meta_t, batch_size = batch_size)
v_gen = data_generator(meta_v, batch_size = batch_size)

In [29]:
# Train model.
forecast_model.fit(t_gen, 
                   epochs = 500, 
                   verbose = 1, 
                   validation_data = v_gen, 
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size,
                  )

Epoch 1/500
101/101 [==============================] - 13s 112ms/step - loss: 443.9440 - mean_squared_error: 443.9440 - val_loss: 229.1947 - val_mean_squared_error: 229.1947
Epoch 2/500
101/101 [==============================] - 11s 112ms/step - loss: 347.9968 - mean_squared_error: 347.9968 - val_loss: 210.2433 - val_mean_squared_error: 210.2433
Epoch 3/500
101/101 [==============================] - 11s 113ms/step - loss: 427.1677 - mean_squared_error: 427.1677 - val_loss: 336.3713 - val_mean_squared_error: 336.3713
Epoch 4/500
101/101 [==============================] - 11s 110ms/step - loss: 417.6918 - mean_squared_error: 417.6918 - val_loss: 385.0517 - val_mean_squared_error: 385.0517
Epoch 5/500
101/101 [==============================] - 11s 111ms/step - loss: 444.8734 - mean_squared_error: 444.8734 - val_loss: 379.7544 - val_mean_squared_error: 379.7544
Epoch 6/500
101/101 [==============================] - 11s 112ms/step - loss: 385.5735 - mean_squared_error: 385.5735 - val_loss: 

Epoch 48/500
101/101 [==============================] - 11s 110ms/step - loss: 329.2280 - mean_squared_error: 329.2280 - val_loss: 204.8325 - val_mean_squared_error: 204.8325
Epoch 49/500
101/101 [==============================] - 11s 110ms/step - loss: 322.1095 - mean_squared_error: 322.1095 - val_loss: 235.2814 - val_mean_squared_error: 235.2814
Epoch 50/500
101/101 [==============================] - 11s 111ms/step - loss: 331.7734 - mean_squared_error: 331.7734 - val_loss: 257.7411 - val_mean_squared_error: 257.7411
Epoch 51/500
101/101 [==============================] - 11s 111ms/step - loss: 327.0166 - mean_squared_error: 327.0166 - val_loss: 257.8368 - val_mean_squared_error: 257.8368
Epoch 52/500
101/101 [==============================] - 11s 111ms/step - loss: 355.0177 - mean_squared_error: 355.0177 - val_loss: 257.7937 - val_mean_squared_error: 257.7937
Epoch 53/500
101/101 [==============================] - 11s 110ms/step - loss: 314.6850 - mean_squared_error: 314.6850 - val_

Epoch 95/500
101/101 [==============================] - 11s 113ms/step - loss: 305.6370 - mean_squared_error: 305.6370 - val_loss: 237.7872 - val_mean_squared_error: 237.7872
Epoch 96/500
101/101 [==============================] - 11s 111ms/step - loss: 256.2441 - mean_squared_error: 256.2441 - val_loss: 325.1250 - val_mean_squared_error: 325.1250
Epoch 97/500
101/101 [==============================] - 11s 112ms/step - loss: 244.5465 - mean_squared_error: 244.5465 - val_loss: 267.6264 - val_mean_squared_error: 267.6264
Epoch 98/500
101/101 [==============================] - 11s 113ms/step - loss: 251.6413 - mean_squared_error: 251.6413 - val_loss: 399.6370 - val_mean_squared_error: 399.6370
Epoch 99/500
101/101 [==============================] - 11s 113ms/step - loss: 271.3947 - mean_squared_error: 271.3947 - val_loss: 258.0857 - val_mean_squared_error: 258.0857
Epoch 100/500
101/101 [==============================] - 11s 111ms/step - loss: 243.1299 - mean_squared_error: 243.1299 - val

101/101 [==============================] - 11s 112ms/step - loss: 136.7203 - mean_squared_error: 136.7203 - val_loss: 402.0381 - val_mean_squared_error: 402.0381
Epoch 188/500
101/101 [==============================] - 11s 114ms/step - loss: 139.9059 - mean_squared_error: 139.9059 - val_loss: 217.8896 - val_mean_squared_error: 217.8896
Epoch 189/500
101/101 [==============================] - 11s 113ms/step - loss: 148.8888 - mean_squared_error: 148.8888 - val_loss: 194.1525 - val_mean_squared_error: 194.1525
Epoch 190/500
101/101 [==============================] - 11s 112ms/step - loss: 140.2042 - mean_squared_error: 140.2042 - val_loss: 318.3857 - val_mean_squared_error: 318.3857
Epoch 191/500
101/101 [==============================] - 11s 113ms/step - loss: 151.4960 - mean_squared_error: 151.4960 - val_loss: 281.1996 - val_mean_squared_error: 281.1996
Epoch 192/500
101/101 [==============================] - 11s 114ms/step - loss: 145.8223 - mean_squared_error: 145.8223 - val_loss: 23

Epoch 280/500
101/101 [==============================] - 11s 114ms/step - loss: 93.4776 - mean_squared_error: 93.4776 - val_loss: 235.9167 - val_mean_squared_error: 235.9167
Epoch 281/500
101/101 [==============================] - 11s 114ms/step - loss: 94.1084 - mean_squared_error: 94.1084 - val_loss: 166.8974 - val_mean_squared_error: 166.8974
Epoch 282/500
101/101 [==============================] - 11s 112ms/step - loss: 103.6996 - mean_squared_error: 103.6996 - val_loss: 244.5537 - val_mean_squared_error: 244.5537
Epoch 283/500
101/101 [==============================] - 11s 113ms/step - loss: 97.5821 - mean_squared_error: 97.5821 - val_loss: 373.7126 - val_mean_squared_error: 373.7126
Epoch 284/500
101/101 [==============================] - 11s 113ms/step - loss: 102.8815 - mean_squared_error: 102.8815 - val_loss: 206.1993 - val_mean_squared_error: 206.1993
Epoch 285/500
101/101 [==============================] - 11s 113ms/step - loss: 99.9134 - mean_squared_error: 99.9134 - val_lo

101/101 [==============================] - 12s 115ms/step - loss: 84.4465 - mean_squared_error: 84.4465 - val_loss: 254.4992 - val_mean_squared_error: 254.4992
Epoch 328/500
101/101 [==============================] - 12s 115ms/step - loss: 86.5419 - mean_squared_error: 86.5419 - val_loss: 200.2990 - val_mean_squared_error: 200.2990
Epoch 329/500
101/101 [==============================] - 12s 115ms/step - loss: 80.3608 - mean_squared_error: 80.3608 - val_loss: 211.2236 - val_mean_squared_error: 211.2236
Epoch 330/500
101/101 [==============================] - 12s 115ms/step - loss: 83.6231 - mean_squared_error: 83.6231 - val_loss: 221.8524 - val_mean_squared_error: 221.8524
Epoch 331/500
101/101 [==============================] - 11s 114ms/step - loss: 83.4936 - mean_squared_error: 83.4936 - val_loss: 256.8626 - val_mean_squared_error: 256.8626
Epoch 332/500
101/101 [==============================] - 12s 116ms/step - loss: 84.1047 - mean_squared_error: 84.1047 - val_loss: 201.4485 - val

101/101 [==============================] - 11s 111ms/step - loss: 61.0587 - mean_squared_error: 61.0587 - val_loss: 168.1682 - val_mean_squared_error: 168.1682
Epoch 422/500
101/101 [==============================] - 11s 110ms/step - loss: 63.2163 - mean_squared_error: 63.2163 - val_loss: 307.3358 - val_mean_squared_error: 307.3358
Epoch 423/500
101/101 [==============================] - 11s 111ms/step - loss: 64.1031 - mean_squared_error: 64.1031 - val_loss: 294.1678 - val_mean_squared_error: 294.1678
Epoch 424/500
101/101 [==============================] - 11s 112ms/step - loss: 62.8870 - mean_squared_error: 62.8870 - val_loss: 248.8612 - val_mean_squared_error: 248.8612
Epoch 425/500
101/101 [==============================] - 11s 112ms/step - loss: 64.0409 - mean_squared_error: 64.0409 - val_loss: 217.7764 - val_mean_squared_error: 217.7764
Epoch 426/500
101/101 [==============================] - 11s 111ms/step - loss: 58.9700 - mean_squared_error: 58.9700 - val_loss: 227.2755 - val

In [30]:
# Save trained model.
forecast_model.save('Models/005')

INFO:tensorflow:Assets written to: Models/005\assets


INFO:tensorflow:Assets written to: Models/005\assets


#### Examine how model is working

In [31]:
# Get data from one chip
sample_input = np.load('Sample_Dataset/val/features/2718135.npy')

# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
a = tf.expand_dims(a, axis = -1)
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)

# Get data from one chip
sample_output = np.load('Sample_Dataset/val/labels/2642208.npy')

# Pad to even number of pixels
b = np.pad(sample_output, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
#b = tf.expand_dims(b, axis = -1)
# Resize to include a batch dimension.
#b = tf.expand_dims(b, axis = 0)
b.sum()

8

In [32]:
pred_eval = forecast_model.predict(a)
pred_eval[0][0]

1/1 [==============================] - 0s 157ms/step


0.79931796

In [34]:
f_list = [
    '2131880.npy',
    '2149116.npy',
    '2164143.npy',
    '2200623.npy',
    '2231628.npy',
    '2249483.npy',
    '2251208.npy',
    '2290512.npy',
    '2315187.npy',
    '2327441.npy'
]

for x in range(0, 10):
    a = np.load('Sample_Dataset/val/features/' + f_list[x])
    a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
    a = tf.expand_dims(a, axis = -1)
    a = tf.expand_dims(a, axis = 0)
    b = np.load('Sample_Dataset/val/labels/' + f_list[x])
    pred_eval = forecast_model.predict(a)
    print(b.sum())
    print(pred_eval[0][0])
    print()

1/1 [==============================] - 0s 15ms/step
0.0
0.79931796

1/1 [==============================] - 0s 15ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
2.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

1/1 [==============================] - 0s 14ms/step
2.0
0.79931796

1/1 [==============================] - 0s 14ms/step
0.0
0.79931796

